# Single Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch as th

from stable_baselines3 import A2C as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

## Init. ENV and Model

In [2]:
env = gym.make('CartPole-v1')
model = ALGO(
    "MlpPolicy",
    env
)

## Functions to Evaluate Model and Train Model within Thread

In [3]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 10
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [4]:
def train(model, timesteps):
    # print('Starting Training')
    model.learn(total_timesteps=timesteps)
    # print('Completed Training')

## Initial Evaluation

In [5]:
evaluate(model, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 9.4


## Train for 1K Steps and Evaluate

In [6]:
# Train MT Model 1
t1 = threading.Thread(target=train, args=(model, 10_00))

# starting thread
t1.start()

# wait until thread is completely executed
t1.join()


# model_trained.learn(total_timesteps=10_00)
evaluate(model, env)

Type  Mean reward: 9.479999999999999


## Apply Gradient and Evaluate

In [7]:
for i in range(100):
    print('Train Iter: ', i, end="| ")

    # Train MT Model 1
    t1 = threading.Thread(target=train, args=(model, 10_0))

    # starting thread
    t1.start()
    
    # wait until thread is completely executed
    t1.join()
    
    evaluate(model, env, 'Updated Model', verbose=0)

Train Iter:  0| Type Updated Model Mean reward: 9.239999999999998
Train Iter:  1| Type Updated Model Mean reward: 9.370000000000001
Train Iter:  2| Type Updated Model Mean reward: 9.43
Train Iter:  3| Type Updated Model Mean reward: 9.35
Train Iter:  4| Type Updated Model Mean reward: 9.379999999999999
Train Iter:  5| Type Updated Model Mean reward: 9.36
Train Iter:  6| Type Updated Model Mean reward: 9.38
Train Iter:  7| Type Updated Model Mean reward: 13.569999999999999
Train Iter:  8| Type Updated Model Mean reward: 21.43
Train Iter:  9| Type Updated Model Mean reward: 42.08
Train Iter:  10| Type Updated Model Mean reward: 34.74
Train Iter:  11| Type Updated Model Mean reward: 43.790000000000006
Train Iter:  12| Type Updated Model Mean reward: 33.489999999999995
Train Iter:  13| Type Updated Model Mean reward: 38.489999999999995
Train Iter:  14| Type Updated Model Mean reward: 41.519999999999996
Train Iter:  15| Type Updated Model Mean reward: 45.260000000000005
Train Iter:  16| Typ

In [8]:
model.get_parameters()

{'policy': OrderedDict([('mlp_extractor.policy_net.0.weight',
               tensor([[ 0.1500, -0.0324,  0.2133, -0.1932],
                       [ 0.1692, -0.0920,  0.1107,  0.1904],
                       [-0.1522, -0.3870,  0.0176,  0.0113],
                       [-0.0551,  0.1407, -0.1030,  0.2227],
                       [ 0.0684,  0.0230, -0.0655, -0.3296],
                       [-0.1282,  0.1884,  0.0453, -0.3626],
                       [ 0.0732, -0.3021,  0.3725,  0.2297],
                       [-0.3927, -0.0301, -0.2671, -0.0933],
                       [ 0.0066,  0.0206, -0.0353, -0.1565],
                       [-0.1811,  0.2460,  0.1564, -0.0930],
                       [ 0.0873, -0.1316,  0.3418,  0.1872],
                       [-0.2673, -0.0053, -0.1599, -0.2595],
                       [ 0.1174,  0.1037,  0.1418,  0.0811],
                       [ 0.0995, -0.1148,  0.1586, -0.0291],
                       [ 0.1537,  0.1314,  0.2342,  0.0685],
                       

In [9]:
model.save('a2c_lunar_singleproc')

In [10]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_singleproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

TypeError: Object of type Tensor is not JSON serializable

In [ ]:
model_loaded = ALGO(
    "MlpPolicy",
    env
)

evaluate(model_loaded,env, verbose=1)

new_params = all_params
loaded_pol_params = new_params['policy']
for key in loaded_pol_params.keys():
    loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

new_params['policy'] = loaded_pol_params

model_loaded.set_parameters(new_params)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)